### asf_search - Downloading Data
`asf_search` provides many ways to find data, but equally important is the ability to download that data. Fortunately, `asf_search` provides a simple interface through which to download data, using a variety of authentication methods.
***
## Before You Start
The steps outlined in this demonstration assume `asf_search` is available in your environment. For guidance on installing `asf_search`, [begin here](./1-Basic_Overview.ipynb#Before-You-Start).

Additionally, this section expects you to have an [Earthdata Login](https://urs.earthdata.nasa.gov/) account with the appropriate applications authorized, EULAs signed, and profile fields set. The easiest way to check that your EDL account is in order is to simply go to [Vertex](https://search.asf.alaska.edu) and download a product.

Lastly, the examples in this notebook assume a few directories exist, namely `./downloads`, `./downloads1`, `./downloads2`, and `./downloads3`. You can create them yourself, or run the following code block:

In [ ]:
from pathlib import Path
dirs = ['downloads', 'downloads1', 'downloads2', 'downloads3']
for d in dirs:
    Path(d).mkdir(exist_ok=True)

***
## ASFSession

Because downloading any product in the ASF archive requires authentication, `asf_search` provides the `ASFSession` class, a subclass of `Session` with a few specific methods added to make authentication straightforward.

Using .netrc credentials is the preferred method for authentication. For more information, see the [session authentication documentation](https://docs.asf.alaska.edu/asf_search/downloading/#session-authentication)

A new, unauthenticated session can be created, although the authentication methods listed below allow chaining for direct creation of an authenticated session.

In [ ]:
import asf_search as asf

session = asf.ASFSession()

### `auth_with_creds()`
This authentication method accepts a username and password and establishes an authentication session with EDL and ASF.

In [ ]:
import getpass
username = input('Username:')
password = getpass.getpass('Password:')

try:
    user_pass_session = asf.ASFSession().auth_with_creds(username, password)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

### `auth_with_token()`
This authentication method accepts an EDL Token which is then included as part of an `Authorization: Bearer` header on any downloads using this session. To generate an EDL Token, [sign in to EDL](https://urs.earthdata.nasa.gov/home), select the "Generate Token" tab, and then click the green "Generate Token" button. The token can then be copied and used below.
  
__Note:__ While it is extremely convenient, not all datapool hosts are compatible with this authentication method yet.

In [ ]:
import getpass
token = getpass.getpass('EDL Token:')

token_session = asf.ASFSession().auth_with_token(token)

### `auth_with_cookiejar()`
This method accepts an `http.cookiejar` compatible object, such as a previously authenticated session stored for later re-use.

For this demonstration, we will make use of the cookiejar from one of the previously authenticated sessions above:

In [ ]:
cookiejar = user_pass_session.cookies

It is to be assumed that this cookiejar is perhaps saved to a file, later loaded, etc. At that time, a new ASFSession can be instantiated using the cookiejar. While it is not required to use this method to reload the session, it can simplify exception handling and EDL/ASF-specific auth processes, and allows a normalized use of `ASFSession` in all cases:

In [ ]:
cookiejar_session = asf.ASFSession().auth_with_cookiejar(cookiejar)

***
## Downloading
[View this search in Vertex](https://search.asf.alaska.edu/#/?dataset=UAVSAR&productTypes=METADATA&resultsLoaded=true&zoom=8.090&center=-90.488,28.359&polygon=POLYGON((-91.97%2028.78,-88.85%2028.78,-88.85%2030.31,-91.97%2030.31,-91.97%2028.78)))
  
With authentication handled, we can now begin downloading products. First, we will need some search results to work with:

In [ ]:
results = asf.geo_search(
    intersectsWith='POLYGON((-91.97 28.78,-88.85 28.78,-88.85 30.31,-91.97 30.31,-91.97 28.78))',
    platform=asf.PLATFORM.UAVSAR,
    processingLevel=asf.PRODUCT_TYPE.METADATA,
    maxResults=250)

print(f'{len(results)} results found')

***
## Downloading Single Products
To download a single `ASFProduct`, simply call its `download()` method, passing in a previously-authenticated session, a path, and optionally a filename. If no filename is provided, the default is to use the filename of the product iself, as described in `properties['fileName']`.

In [ ]:
from os import listdir

results[0].download(path='./downloads1', session=user_pass_session)

listdir('./downloads1')

Some results may be stored as zip files. To download only part of a single `ASFProduct`'s zip, call its `remotezip()` method, passing in a previously-authenticated session. It should return a `RemoteZip` object, which provides functionality to download parts of the `ASFProduct`'s zip archive. Below is an example of using a `RemoteZip` object to download all .tiff files from a single product.

In [ ]:
results_with_zips = asf.search(platform=asf.constants.PLATFORM.SENTINEL1, processingLevel=asf.constants.PRODUCT_TYPE.GRD_HD, maxResults=250)

with results_with_zips[0].remotezip(session=user_pass_session) as z:
    file_paths = [file.filename for file in z.filelist if file.filename.endswith('.tiff')]

    print(f'found {len(file_paths)} tiff files in zip')

    for file_path in file_paths:
        z.extract(file_path, path='./downloads1')

listdir('./downloads1')

For more information on remotezip functionality, see https://github.com/gtsystem/python-remotezip

***
## Downloading Multiple Products
More often than not, we want to download an entire set of search results rather than just a single product. `ASFSearchResults` provides this functionality similarly to `ASFProduct` via the identically-named `download()` method, albeit with two key differences: filenames always use the default behavior, and downloads can occur in parallel. If a particular file already exists, a `UserWarning` will be emitted, and the file will be skipped.

In [ ]:
results[0:10].download(path='./downloads1', session=user_pass_session)
listdir('./downloads1')

While the above example downloads each file in sequence by default, it is often more performant to download multiple files in parallel. With that in mind, `ASFSearchResults.download()` allows setting a maximum number downloads to run in parallel:

In [ ]:
results.download(path='./downloads2', session=user_pass_session, processes=50)

listdir('./downloads2')

***
## Downloading Arbitrary URLs
Lastly, it may occur that you have a list of product URLs you wish to download, but have not arrived at that list through `asf_search`. Perhaps you have a service in the cloud and it's convenient to just copy/paste a list of URLs from some external process. In that case, `asf_search` exposes its download functionality more directly, through `download_urls()`. This function takes a list of arbitrary URLs, a path, an authenticated session, and optionally a number of downloads to run in parallel:

In [ ]:
urls = [
    'https://datapool.asf.alaska.edu/METADATA/UA/aleutn_06005_09051_003_090723_L090_CX_01.ann',
    'https://datapool.asf.alaska.edu/METADATA/UA/aleutn_06004_09051_004_090723_L090_CX_01.ann',
    'https://datapool.asf.alaska.edu/METADATA/UA/aleutn_04701_09051_005_090723_L090_CX_01.ann',
    'https://datapool.asf.alaska.edu/METADATA/UA/aleutn_23301_09050_001_090722_L090_CX_01.ann',
    'https://datapool.asf.alaska.edu/METADATA/UA/aleutn_19802_11054_001_110802_L090_CX_01.ann']

asf.download_urls(urls=urls, path='./downloads3', session=user_pass_session, processes=5)

listdir('./downloads3')

***
## Summary
A complete, basic example of downloading search results:

In [ ]:
from os import listdir
import getpass
username = input('Username:')
password = getpass.getpass('Password:')

import asf_search as asf

session = asf.ASFSession().auth_with_creds(username=username, password=password)

results = asf.geo_search(
    intersectsWith='POLYGON((-91.97 28.78,-88.85 28.78,-88.85 30.31,-91.97 30.31,-91.97 28.78))',
    platform=asf.PLATFORM.UAVSAR,
    processingLevel=asf.PRODUCT_TYPE.METADATA,
    maxResults=20)

results.download(
    path='./downloads',
    session=session,
    processes=10)

listdir('./downloads')

***
Next: [Closing](./6-Outro.md)